In [401]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [402]:
from functools import wraps
import inspect

from htools import hdir, magics

In [418]:
def autoinit(cls):
    def wrapped(*args, **kwargs):
        bound = inspect.signature(cls.__init__)
#         .bind(*args, **kwargs)
        print(bound)
#         for k, v in kwargs.items():
#             setattr(cls, k, v)
        return cls
    return wrapped

In [419]:
@autoinit
class Foo:
    
    def __init__(self, a, b, c=3, d='d', e=[1, 2, 3]):
        pass

In [420]:
f = Foo(1, 'b')
f

(self, a, b, c=3, d='d', e=[1, 2, 3])


__main__.Foo

## decorator: enforce types

In [9]:
def typecheck(**types):
    def decorator(func):
        @wraps(func)
        def wrapped(*args, **kwargs):
            fargs = inspect.signature(wrapped).bind(*args, **kwargs).arguments
            for k, v in types.items():
                if k in fargs and not isinstance(fargs[k], v):
                    raise TypeError(
                        f'{k} must be {str(v)}, not {type(fargs[k])}.'
                    )
            return func(*args, **kwargs)
        return wrapped
    return decorator

In [10]:
class Bar:
    
    def __init__(self, a:int, b:str, c:list=[1,2,3,4], d:float=0.0):
        self.a = a
        self.b = b
        self.c = c
        self.d = d

In [11]:
dec = typecheck(a=int, b=float, c=str)
dec(Bar.__init__)(Bar, a=1, b=2.0, c='3')

In [12]:
class Foo:
    
    @typecheck(a=int, b=str, c=int, e=list)
    def __init__(self, a, b, c=3, d='d', e=[1, 2, 3]):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        
    @typecheck(x=(int, float))
    def compute(self, x, label='abc'):
        print(label)
        return self.a * self.c / x

In [13]:
f = Foo(1, 'b', 6, e=[3, 4])

In [14]:
print(f.compute(14))
print(f.compute(3.0, label=4444))
print(f.compute('a'))

abc
0.42857142857142855
4444
2.0


TypeError: x must be (<class 'int'>, <class 'float'>), not <class 'str'>.

In [15]:
f = Foo(1, b='b', d=4)

In [16]:
f = Foo(1, 'b', [1,3])

TypeError: c must be <class 'int'>, not <class 'list'>.

## decorator: enforce type annotations

In [17]:
def enforce_types(func):
    """Wrapper for @typecheck decorator that infers types based off of 
    annotations.
    
    Parameters
    ----------
    func: function
        The function to decorate.
        
    Examples
    --------
    In the example below, notice that type annotations can be a single type,
    a tuple of types, or excluded entirely. Annotations regarding the returned
    value are not checked.
    
    @enforce_types
    def process(x:float, y:(int, float), z, iters:int=5, verbose:bool=True):
        print(f'z = {z}')
        for i in range(iters):
            if verbose: print(f'Iteration {i}...')
            x *= y
        return x
        
    >>> process(3.1, 4.5, 0, 2.0)
    TypeError: iters must be <class 'int'>, not <class 'float'>.
    
    >>> process(3.1, 4, 'a', 1, False)
    z = a
    12.4
    """
    arg_types = {k: v.annotation 
                 for k, v in inspect.signature(func).parameters.items()
                 if not v.annotation == inspect._empty}
    return typecheck(**arg_types)(func)

In [18]:
@enforce_types
def process(x:float, y:(int, float), z, iters:int=5, verbose:bool=True):
    print(f'z = {z}')
    for i in range(iters):
        if verbose: print(f'Iteration {i}...')
        x *= y
    return x

In [19]:
process(3.1, 4, 'a', 2.0)

TypeError: iters must be <class 'int'>, not <class 'float'>.

In [20]:
process(3.1, 4, 'a', 1, False)

z = a


12.4

In [22]:
@enforce_types
def foobar(a:int, b:str, c:list, d:float=0.0, e:bool=False):
    """Return tuple of arguments."""
    return a, b, c, d, e

In [29]:
foobar(1, 'b', ['c'], 0.0, True)

(1, 'b', ['c'], 0.0, True)

In [30]:
foobar(a=1, b='b', c=[1], d=1.0, e=True)

(1, 'b', [1], 1.0, True)

In [31]:
foobar(1, 'b', [1], d=1.0, e=True)

(1, 'b', [1], 1.0, True)

In [32]:
foobar(a=1, b='b', c=[1], d=False)

TypeError: d must be <class 'float'>, not <class 'bool'>.

In [33]:
foobar(1, 2, ['c'], e=True)

TypeError: b must be <class 'str'>, not <class 'int'>.

In [34]:
foobar(1, '2', 'c', e=True)

TypeError: c must be <class 'list'>, not <class 'str'>.

In [91]:
class EnforceTypes:
    
    def __init__(self, func):
        print('ET init')
        self.func = func
        self.arg_types = {k: v.annotation for k, v
                          in inspect.signature(func).parameters.items()
                          if not v.annotation == inspect._empty}
        self.check = typecheck(**self.arg_types)
        
    def __call__(self, *args, **kwargs):
        print('ET call')
        return self.check(self.func)(*args, **kwargs)

In [92]:
@EnforceTypes
def foobar2(a:int, b:str, c:list, d:float=0.0, e:bool=False):
    """Return tuple of arguments."""
    return a, b, c, d, e

ET init


In [93]:
foobar2(1, 'b', [1, 3])

ET call


(1, 'b', [1, 3], 0.0, False)

In [94]:
class Foo:
    
    @EnforceTypes
    def __init__(self, a, b, c=3, d='d', e=[1, 2, 3]):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        
    @EnforceTypes
    def compute(self, x, label='abc'):
        print(label)
        return self.a * self.c / x

ET init
ET init


In [96]:
f = Foo(a=1, b=2)

ET call


TypeError: missing a required argument: 'self'

In [87]:
%%race -n 100 -r 100
_ = foobar(a=1, b='b', c=[1], d=1.0, e=True)
_ = foobar2(a=1, b='b', c=[1], d=1.0, e=True)

In [39]:
%%race -n 100 -r 100
_ = foobar(1, 'b', c=[1], e=True)
_ = foobar2(1, 'b', c=[1], e=True)

The slowest run took 4.89 times longer than the fastest. This could mean that an intermediate result is being cached.
31.2 µs ± 14 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
33.8 µs ± 7 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


## Try to combine into 1 decorator that can be called with or without parens

In [101]:
def f(func=None, **kwargs):
    if func:
        print(func)
    else:
        print('no func')

In [102]:
f(print)

<built-in function print>


In [109]:
from functools import partial

In [276]:
def typecheck(func_=None, **types):
    """
    Parameters
    ----------
    func_: function
        The function to decorate. When using decorator with 
        manually-specified types, this is None. Underscore is used so that
        `func` can still be used as a valid keyword argument for the wrapped 
        function.
    types: type
        Optional way to specify variable types.
        
    Examples
    --------
    In the first example, we specify types directly in the decorator. Notice
    that they can be single types or tuples of types. You can choose to 
    specify types for all arguments or just a subset.
    
    @typecheck(x=float, y=(int, float), iters=int, verbose=bool)
    def process(x, y, z, iters=5, verbose=True):
        print(f'z = {z}')
        for i in range(iters):
            if verbose: print(f'Iteration {i}...')
            x *= y
        return x
    
    >>> process(3.1, 4.5, 0, 2.0)
    TypeError: iters must be <class 'int'>, not <class 'float'>.
    
    >>> process(3.1, 4, 'a', 1, False)
    z = a
    12.4
    
    Alternatively, you can let the decorator infer types using annotations
    in the function that is to be decorated. The example below behaves 
    equivalently to the explicit example shown above. Note that annotations
    regarding the returned value are ignored.
    
    @typecheck
    def process(x:float, y:(int, float), z, iters:int=5, verbose:bool=True):
        print(f'z = {z}')
        for i in range(iters):
            if verbose: print(f'Iteration {i}...')
            x *= y
        return x
        
    >>> process(3.1, 4.5, 0, 2.0)
    TypeError: iters must be <class 'int'>, not <class 'float'>.
    
    >>> process(3.1, 4, 'a', 1, False)
    z = a
    12.4
    """
    # Case 1: Pass keyword args to decorator specifying types.
    if not func_:
        return partial(typecheck, **types)
    # Case 2: Infer types from annotations. Skip if Case 1 already occurred.
    elif not types:
        types = {k: v.annotation 
                 for k, v in inspect.signature(func_).parameters.items()
                 if not v.annotation == inspect._empty}
    
    @wraps(func_)
    def wrapped(*args, **kwargs):
        fargs = inspect.signature(wrapped).bind(*args, **kwargs).arguments
        for k, v in types.items():
            if k in fargs and not isinstance(fargs[k], v):
                raise TypeError(
                    f'{k} must be {str(v)}, not {type(fargs[k])}.'
                )
        return func_(*args, **kwargs)
    return wrapped

In [277]:
@typecheck(x=float, y=(int, float), iters=int, verbose=bool)
def process(x, y, z, iters=5, verbose=True):
    """Process docstring."""
    print(f'z = {z}')
    for i in range(iters):
        if verbose: print(f'Iteration {i}...')
        x *= y
    return x

In [278]:
process.__doc__

'Process docstring.'

In [275]:
process(3.1, 4, 'a', 1, False)

z = a


12.4

In [274]:
process(3.1, 4.5, 0, 2.0)

TypeError: iters must be <class 'int'>, not <class 'float'>.

In [252]:
@typecheck(a=int, c=str, d=list)
def foo3(a, b=3, c='d', d=[1, 2, 3]):
    return d, c, b, a

In [255]:
foo3(2)

([1, 2, 3], 'd', 3, 2)

In [257]:
foo3(2.0)

TypeError: a must be <class 'int'>, not <class 'float'>.

In [258]:
foo3(1, 'a', 0)

TypeError: c must be <class 'str'>, not <class 'int'>.

In [259]:
foo3(1, 4.0, c='c', d=[9])

([9], 'c', 4.0, 1)

In [260]:
@typecheck(b=(int, float), c=str)
def foo4(a, b=3, c='d', d=[1, 2, 3]):
    return d, c, b, a

In [261]:
foo4('a')

([1, 2, 3], 'd', 3, 'a')

In [262]:
foo4(3, 2.0)

([1, 2, 3], 'd', 2.0, 3)

In [263]:
@typecheck
def foo5(a:int, b=3, c:str='d', d:list=[1, 2, 3]):
    return d, c, b, a

In [264]:
foo5(1)

([1, 2, 3], 'd', 3, 1)

In [265]:
foo5('a')

TypeError: a must be <class 'int'>, not <class 'str'>.

In [266]:
@typecheck
def foo5(a:str, b=3, c:(int, float)=4.0, d=[1, 2, 3]):
    return d, c, b, a

In [267]:
foo5(a='a')

([1, 2, 3], 4.0, 3, 'a')

In [268]:
foo5('a', 33, c=9, d=0)

(0, 9, 33, 'a')

In [269]:
foo5('a', 'b', 'c', 'd')

TypeError: c must be (<class 'int'>, <class 'float'>), not <class 'str'>.

In [291]:
class Foo:
    
    @typecheck
    def __init__(self, a:str, b:int, c:(int, float)=3, d='d', e:list=[1, 2, 3]):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        
    @typecheck(x=int, label=str)
    def compute(self, x, label='abc'):
        print(label)
        return self.b * self.c / x

In [293]:
f = Foo('a', 3, 33)

In [294]:
f.compute(2, 'a')

a


49.5

In [295]:
f = Foo('a', 0, d=True, e=False)

TypeError: e must be <class 'list'>, not <class 'bool'>.

In [ ]:
# Just thinking through call process, basically pseudocode.

# Explicit types. First pass in types to create decorator, then call on func, then call wrapped func.
# dec = typecheck(a=int, b=str)  
# decorated = dec(func)  
# decorated(*args, **kwargs)  

# Inferred types. First pass func to typecheck, then call on args and kwargs. 
# decorated = typecheck(func)  
# decorated(*args, **kwargs)  

In [299]:
from typing import List

In [302]:
isinstance(['a', 'b'], List[str])

TypeError: Subscripted generics cannot be used with class and instance checks

In [344]:
# Book example
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        print(cls, clsname, bases, clsdict)
        order = []
        for name, value in cls.__init__.__code__.co_varnames:
            print(name, value)
#             if isinstance(value, (int, str, float, bool)): continue
#             value._name = name
#             order.append(name)
#         d['_order'] = order
        return type.__new__(cls, clsname, bases, d)

In [345]:
# Book example
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        print('IN AS_CSV')    
        return ','.join(str(getattr(self,name)) for name in self._order)

<class '__main__.OrderedMeta'> Structure () {'__module__': '__main__', '__qualname__': 'Structure', 'as_csv': <function Structure.as_csv at 0x10b556730>}


AttributeError: 'wrapper_descriptor' object has no attribute '__code__'

In [340]:
# Example use
class Stock(Structure):
    x = 3
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

<class '__main__.OrderedMeta'> Stock (<class '__main__.Structure'>,) {'__module__': '__main__', '__qualname__': 'Stock', 'x': 3, '__init__': <function Stock.__init__ at 0x10b45ad08>}


In [341]:
s = Stock('n', 's', 'p')

In [342]:
hdir(s)

{'as_csv': 'method',
 'name': 'attribute',
 'price': 'attribute',
 'shares': 'attribute',
 'x': 'attribute'}

In [343]:
s.as_csv()

IN AS_CSV


AttributeError: 'Stock' object has no attribute '_order'

In [325]:
s._order

['__init__']

## try to rebuild autoinit as decorator

In [421]:
def autoinit(cls):
    @wraps(cls)
    def wrapped(*args, **kwargs):
        print(args, kwargs)
        obj = cls.__new__(cls)
        bound = inspect.signature(cls.__init__).bind(obj, *args, **kwargs)
        bound.apply_defaults()
        
        arg_str = cls.__name__ + '('
        for k, v in bound.arguments.items():
            if k == 'self': continue
            setattr(obj, k, v)
            arg_str += f'{k}={v}, '
            
        def __repr__(obj):
            arg_str = ', '.join(f'{k}={v}' 
                                for k, v in bound.arguments.items())
            return obj.__name__
        
        obj.__repr__ = __repr__
        return obj
    return wrapped

In [422]:
@autoinit
class Data:
    
    def __init__(self, a, b=3, c=True, **kwargs):
#         self.a = a
#         self.b = b
#         self.c = c
        pass
    
#     def __repr__(self):
#         return f'Data({self.a}, {self.b}, {self.c})'

In [ ]:
Data.__new__()

In [423]:
d = Data(91, z=44)
d

(91,) {'z': 44}


In [424]:
d.__dict__

{'a': 91,
 'b': 3,
 'c': True,
 'kwargs': {'z': 44},
 '__repr__': <function __main__.autoinit.<locals>.wrapped.<locals>.__repr__(obj)>}

In [427]:
d.__repr__(d)

AttributeError: 'Data' object has no attribute '__name__'

In [377]:
bound = inspect.signature(foo5).bind('a', c=6)
hdir(bound)

{'apply_defaults': 'method',
 'args': 'attribute',
 'arguments': 'attribute',
 'kwargs': 'attribute',
 'signature': 'attribute'}

In [379]:
%%talk
bound.args
bound.kwargs
bound.arguments

('a',)

{'c': 6}

OrderedDict([('a', 'a'), ('c', 6)])

In [388]:
bound.apply_defaults()

In [389]:
bound

<BoundArguments (a='a', b=3, c=6, d=[1, 2, 3])>

## Alternate inheritance method - trying to avoid need to pass locals()

In [3]:
from contextlib import contextmanager
import inspect
import sys

from htools import *

In [366]:
# Temp 2 to distinguish from current implementation.

# Note: likely because of frame hack, this version is ~8x slower. However, in
# the example below this still only took 30-40 microseconds 
# (i.e. 3e-5 seconds), so in practice this probably isn't a huge concern in 
# most scenarios.
class AutoInit2:
    def __init__(self):
        # Calculate how many frames to go back to get child class.
        frame_idx = type(self).__mro__.index(AutoInit2) # TODO: rm 2 if shipping
        attrs = {k: v for k, v in sys._getframe(frame_idx).f_locals.items()
                  if not k.startswith('__')}
        attrs.pop('self')
        bound = inspect.signature(self.__class__.__init__)\
                       .bind_partial(**attrs)
        
        # Flatten dict so kwargs are not listed as their own argument.
        bound.arguments.update(
            bound.arguments.pop('kwargs', {}).get('kwargs', {})
        )
        self._init_keys = set(bound.arguments.keys())
        for k, v in bound.arguments.items():
            setattr(self, k, v)

    def __repr__(self):
        fstrs = (f'{k}={repr(getattr(self, k))}' for k in self._init_keys)            
        return f'{self.__class__.__name__}({", ".join(fstrs)})'

In [367]:
class Niece(AutoInit2):
    def __init__(self):
        super().__init__()
    def niecemethod(self):
        print('niece call niecemethod')

In [368]:
class Mixin:
    def mix(self):
        print('Mixin call self.mix')

In [369]:
class Multi(Niece, Mixin):
    def __init__(self, a, b, c=3, **kwargs):
        super().__init__()

In [370]:
class Child(AutoInit2):
    def __init__(self, a, b=3, **kwargs):
        super().__init__()
    def childmethod(self):
        print('child call childmethod')

In [371]:
class ProblemChild(AutoInit):
    b = ReadOnly('b')
    def __init__(self, a, b=3, **kwargs):
        super().__init__(locals())
    def childmethod(self):
        print('child call childmethod')
    @cached_property
    def c(self):
        for i in range(10_000_000):
            pass
        return self.a*self.b

In [372]:
class ProblemChild2(AutoInit2):
    b = ReadOnly('b')
    def __init__(self, a, b=3, **kwargs):
        super().__init__()
    def childmethod(self):
        print('child call childmethod')
    @cached_property
    def c(self):
        for i in range(10_000_000):
            pass
        return self.a*self.b

In [373]:
@contextmanager
def assert_raises(error, verbose=True):
    try:
        yield
    except error as e:
        if verbose: print(f'As expected, got {error.__name__}({e}).')
    except Exception as e:
        raise AssertionError(f'Wrong error raised. Expected {error.__name__},'
                             f' got {type(e).__name__}({e}).') from None
    else:
        raise AssertionError(f'No error raised, expected {error.__name__}.')

In [374]:
n = Niece()
n

Niece()

In [375]:
m = Multi(1, 2, d='d', e='e')
m

Multi(c=3, e='e', d='d', b=2, a=1)

In [376]:
c = Child('a', d='d', e=[1,2,3])
c

Child(e=[1, 2, 3], b=3, d='d', a='a')

In [377]:
p = ProblemChild(2, e=True, d=[56, 55, 44])
p

ProblemChild(e=True, b=3, d=[56, 55, 44], a=2)

In [378]:
with assert_raises(ValueError) as a:
    p.b = 2

AssertionError: Wrong error raised. Expected ValueError, got PermissionError(Attribute is read-only.).

In [379]:
with assert_raises(PermissionError) as a:
    p.a = 2

AssertionError: No error raised, expected PermissionError.

In [380]:
with assert_raises(PermissionError) as a:
    p.b = 2

As expected, got PermissionError(Attribute is read-only.).


In [381]:
p.c

6

In [382]:
# Faster because result is cached now.
p.c

6

In [383]:
p2 = ProblemChild2(2, e=True, d=[56, 55, 44])
p2

ProblemChild2(e=True, b=3, d=[56, 55, 44], a=2)

In [384]:
with assert_raises(PermissionError) as ar:
    p2.b = 33

As expected, got PermissionError(Attribute is read-only.).


In [385]:
p2.c

6

In [386]:
p3 = ProblemChild2(0, b=44, z=['a', 'b'])
p3

ProblemChild2(z=['a', 'b'], b=44, a=0)

In [387]:
p3.c

0

In [388]:
p3.c

0

In [389]:
with assert_raises(PermissionError) as ar:
    p3.b = 'b'

As expected, got PermissionError(Attribute is read-only.).


In [390]:
with assert_raises(PermissionError) as ar:
    del p3.b

As expected, got PermissionError(Attribute is read-only.).


In [391]:
p3.b

44

In [392]:
p3.__dict__['b'] = 99
p3

ProblemChild2(z=['a', 'b'], b=99, a=0)

In [393]:
def f():
    return ProblemChild(1, e=True, d=[56, 55, 44])
    
def main():
    m = Multi(11, 111, e='e')
    p = f()
    return m, p

m, p = main()

In [394]:
m

Multi(e='e', b=111, c=3, a=11)

In [395]:
p

ProblemChild(e=True, b=3, d=[56, 55, 44], a=1)

In [396]:
%%race -n 100 -r 50
p = ProblemChild(2, e=True, d=[56, 55, 44])
p2 = ProblemChild2(2, e=True, d=[56, 55, 44])

4.46 µs ± 1.17 µs per loop (mean ± std. dev. of 50 runs, 100 loops each)
31.8 µs ± 5.01 µs per loop (mean ± std. dev. of 50 runs, 100 loops each)


In [397]:
%%race -n 100 -r 50
p = ProblemChild(2, e=True, d=[56, 55, 44], x=True, y=False, z=[1,2], m={})
p2 = ProblemChild2(2, e=True, d=[56, 55, 44], x=True, y=False, z=[1,2], m={})

7.27 µs ± 1.49 µs per loop (mean ± std. dev. of 50 runs, 100 loops each)
The slowest run took 5.41 times longer than the fastest. This could mean that an intermediate result is being cached.
44 µs ± 23.5 µs per loop (mean ± std. dev. of 50 runs, 100 loops each)


## @debug - rm self from args if method

In [14]:
from functools import wraps
import inspect

from htools import hdir

In [25]:
def debug(func=None, prefix='', arguments=True):
    """Decorator that prints information about a function call. Often, this
    will only be used temporarily when debugging. Note that a wrapped function
    that accepts *args will display a signature including an 'args' parameter
    even though it isn't a named parameter, because the goal here is to
    explicitly show which values are being passed to which parameters. This
    does mean that the printed string won't be executable code in this case,
    but that shouldn't be necessary anyway since it would contain the same call
    that just occurred.

    The decorator can be used with or without arguments.

    Parameters
    ----------
    func: function
        Function being decorated.
    prefix: str
        A short string to prepend the printed message with. Ex: '>>>'
    arguments: bool
        If True, the printed message will include the function arguments.
        If False, it will print the function name but not its arguments.

    Examples
    --------
    Occasionally, you might pass arguments to different parameters than you
    intended. Throwing a debug_call decorator on the function helps you check
    that the arguments are matching up as expected. For example, the parameter
    names in the function below have an unexpected order, so you could easily
    make the following call and expect to get 8. The debug decorator helps
    catch that the third argument is being passed in as the x parameter.

    @debug
    def f(a, b, x=0, y=None, z=4, c=2):
        return a + b + c

    >>> f(3, 4, 1)
    CALLING f(a=3, b=4, x=1, y=None, z=4, c=2)
    9

    @debug(prefix='***', arguments=False)
    def f(a, b, x=0, y=None, z=4, c=2):
        return a + b + c

    >>> f(3, 4, 1)
    *** CALLING f()
    9
    """
    if not func:
        if prefix: prefix += ' '
        return partial(debug, prefix=prefix, arguments=arguments)

    @wraps(func)
    def wrapper(*args, **kwargs):
        out_fmt = '\n{}CALLING {}({})'
        arg_strs = ''
        if arguments:
            sig = inspect.signature(wrapper).bind_partial(*args, **kwargs)
            sig.apply_defaults()
            sig.arguments.update(sig.arguments.pop('kwargs', {}))
            # Remove self/cls arg from methods.
            first_key = next(iter(sig.arguments))
            if first_key in ('self', 'cls'): 
                del sig.arguments[first_key]
            arg_strs = (f'{k}={repr(v)}' for k, v in sig.arguments.items())

        # Print call message and return output.
        print(out_fmt.format(prefix, func.__qualname__, ', '.join(arg_strs)))
        return func(*args, **kwargs)

    return wrapper

In [26]:
class Foo: 
    @debug
    def __init__(self, a=6, b:int=1, c:str='c', d=True): 
        self.a, self.b, self.c, self.d = a, b, c, d 
    def walk(self): 
        return 'walk' 
    def _run(self, r:(bool, int)): 
        return r 
    def jump(self, x:bool, y, z:list=[1,2]): 
        return x, y, z 
    def __private(self, p:int=0): 
        return 'private', p 

In [27]:
f = Foo()


CALLING Foo.__init__(a=6, b=1, c='c', d=True)


In [29]:
@debug
def bar(x, y=True):
    return x

In [30]:
bar(3)


CALLING bar(x=3, y=True)


3